# Durr & Hoyer - Minima/Maxima Finding Algorithm


Durr & Hoyer

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from math import pi
import math
import random
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from qiskit import IBMQ, Aer, BasicAer, execute
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.quantum_info import state_fidelity
from qiskit.tools.visualization import circuit_drawer
from qiskit.circuit.library.generalized_gates.mcmt import MCMT
from qiskit.circuit.library.standard_gates.x import XGate, HGate
from qiskit.circuit.library import ZGate
from qiskit.visualization import plot_histogram

The following array will be used throughout the notebook.

In [2]:
arr = [22, 42, 2, 7, 74, 11, 86, 10, 3, 6, 93, 90, 23, 89, 92, 203, 84, 33, 32, 31, 40, 49, 84, 92]
n = len(arr)
print("The number of elements in the array is " + str(n) + ".")

The number of elements in the array is 24.


In [3]:
pairedarr = []
sortedind = []
for i in range(len(arr)):
    pairedarr.append((arr[i], i))
pairedarr.sort()
for i in range(len(pairedarr)):
    sortedind.append(pairedarr[i][1])
#print(sortedind)

def f(x):
    for i in range(len(sortedind)):
        if (sortedind[i] == x):
            return sortedind[:i]
#print(f(0))

### Oracles
The following oracles (AndOracle, ControlledOnIntOracle) are useful to us in the implementation. 

In [4]:
#nn = size of given_circuit
def AndOracle(given_circuit, nn, target_qubit):
    given_circuit.barrier()
    controlled_xgate = XGate().control(nn)
    curlist = list(range(0, nn))
    curlist.append(target_qubit)
    given_circuit.append(controlled_xgate, curlist)
    given_circuit.barrier()

#nn = size of given_circuit excluding the target_qubit
def ControlledOnIntOracle(given_circuit, nn, target_qubit, x, reverse=False):
    
    if (x != 0 and nn < int(math.floor(math.log2(x) + 1))):
        print("Not enough qubits to do ControlledOnIntOracle\n")
        return
    
    given_circuit.barrier()
    xx = x
    counter = 0
    for i in range(nn):
        if (xx%2 == 0):
            given_circuit.x(counter)
        xx = (xx - (xx%2))/2
        counter = counter + 1
    AndOracle(given_circuit, nn, target_qubit)
    
    xx = x
    counter = 0
    for i in range(nn):
        if (xx%2 == 0):
            given_circuit.x(counter)
        xx = (xx - (xx%2))/2
        counter = counter + 1
    given_circuit.barrier()

def DiffusionOperator(given_circuit, nn, name = "DiffusionOperator"):
    
    given_circuit.h(range(nn))
    given_circuit.x(range(nn))
    controlled_gate = ZGate().control(nn - 1)
    given_circuit.append(controlled_gate, list(range(0, nn)))
    given_circuit.x(range(nn))
    given_circuit.h(range(nn))

def GroverOracle(given_circuit, nn, indices_to_mark, name = 'GroverOracle'):
    for i in indices_to_mark:
        ControlledOnIntOracle(given_circuit, nn, nn, i)

def GroverAlgorithm(given_circuit, nn, indices_of_marked_elements):
    
    # Determine r
    r = int(np.floor(np.pi/4*np.sqrt(2**nn/len(indices_of_marked_elements))))
    print(f'{nn} qubits, basis states {indices_of_marked_elements} marked, {r} rounds')
    
    # step 1: apply Hadamard gates on all qubits
    given_circuit.h(range(nn))
    
    # step 2: apply r rounds of the phase oracle and the diffuser
    for _ in range(r):
        GroverOracle(given_circuit, nn, indices_of_marked_elements)
        DiffusionOperator(given_circuit, nn)
        #qc.append(phase_oracle(nn, indices_of_marked_elements), range(n))
        #qc.append(diffuser(nn), range(nn))
        
    # step 3: measure all qubits
    given_circuit.measure(range(nn), range(nn))

## Minimum Oracle

Now, we need to implement the minimum oracle.

That is, an oracle such that given i, returns a superposition of all indices j such that arr[j] < arr[i].

The creation of such an oracle shown here is O(n^2). No choice.

However, the essence of this algorithm does not lie in the creation of the algorithm, more of whether it exists or not.

In [5]:
def MinimumOracle(given_circuit, nn, target_qubit, x, reverse=False, name = "MinimumOracle"):
    print("Called Minimum Oracle with index " + str(x) + ". arr[x] = " + str(arr[x]) + ".")
    print("Corresponding oracle gives " + str(f(x)))
    if (len(f(x)) == 0):
        return False
    GroverAlgorithm(given_circuit, nn, f(x))
    return True

In [6]:
simulator = Aer.get_backend('qasm_simulator')

def converttoint(s):
    ans = 0
    for i in range(len(s)):
        ans = ans + (2**i)*int(s[len(s) - i - 1])
    return ans


#nn is the number of qubits, excluding the ancilla
#x is the number of items in the array
def DurrHoyerAlgorithm(nn, x):
    
    callstooracle = 0
    
    #Step 1: Generates a random index
    y = random.randint(0, x - 1)
    done = True
    
    while (done == True):
        durrhoyer_q = QuantumRegister(nn + 1)
        durrhoyer_c = ClassicalRegister(nn)
        durrhoyer_qc = QuantumCircuit(durrhoyer_q, durrhoyer_c, name = "DurrHoyer Circuit")

        #Step 2: apply Hadamard gate to all
        #durrhoyer_qc.h(range(nn))

        #Step 3: call the oracle
        done = MinimumOracle(durrhoyer_qc, nn, nn, y)
        callstooracle = callstooracle + 1
        if (done == False):
            return (y, callstooracle, counts)
        durrhoyer_qc.measure(range(nn), range(nn))
        counts = execute(durrhoyer_qc, simulator, shots = 200).result().get_counts(durrhoyer_qc)
        sortedcounts = {k: v for k, v in sorted(counts.items(), key=lambda item: item[1])}
        print(sortedcounts)
        print(len(sortedcounts))
        print(list(sortedcounts.keys())[-1])
        curlist = list(sortedcounts.keys())
        yprime = 0
        for i in range(len(curlist)):
            if (converttoint(curlist[-1 - i]) >= x):
                continue
            yprime = converttoint(curlist[-1-i])
            break
    
        print(yprime)

        if (arr[yprime] < arr[y]):
            y = yprime
        del durrhoyer_qc
    
    return (y, callstooracle, counts)

In [7]:
newn = 1 + math.floor(math.log2(n))
(min_index, num_calls, final_count) = DurrHoyerAlgorithm(newn, n)
plot_histogram(final_count)
print("The minimum index is " + str(min_index) + " with value arr[x] of " + str(arr[min_index]) + ".")
print("The number of calls made to the minimum oracle is " + str(num_calls) + ".")

Called Minimum Oracle with index 1. arr[x] = 42.
Corresponding oracle gives [2, 8, 9, 3, 7, 5, 0, 12, 19, 18, 17, 20]
5 qubits, basis states [2, 8, 9, 3, 7, 5, 0, 12, 19, 18, 17, 20] marked, 1 rounds
{'11100': 1, '00100': 2, '10101': 3, '10111': 3, '10001': 3, '01010': 3, '10110': 3, '11010': 3, '11000': 4, '00111': 4, '01111': 4, '11011': 4, '11111': 5, '00001': 5, '00110': 5, '01110': 5, '00010': 5, '11101': 5, '01011': 6, '01101': 6, '10000': 7, '11001': 7, '10010': 7, '11110': 8, '01001': 9, '00000': 9, '10100': 10, '10011': 10, '00011': 11, '00101': 12, '01100': 13, '01000': 18}
32
01000
8
Called Minimum Oracle with index 8. arr[x] = 3.
Corresponding oracle gives [2]
5 qubits, basis states [2] marked, 4 rounds
{'01001': 1, '10011': 1, '00001': 2, '10101': 2, '01010': 2, '00101': 2, '11000': 2, '11100': 2, '01100': 2, '00000': 2, '01101': 2, '01111': 2, '11011': 2, '00110': 3, '10111': 3, '10110': 3, '11010': 3, '11111': 4, '00100': 4, '01110': 4, '11101': 4, '11110': 5, '10100': 5

### References

1. http://cds.cern.ch/record/408677/files/9911082.pdf